In [153]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import datetime

currency_list = ['USDCHF1440',
                 #'GBPUSD1440', 
                 #'EURUSD1440',
    #'USDJPY1440',
    #'USDCAD1440', 
    #'AUDUSD1440',
    #'NZDUSD1440',
                 # 'GBPCHF1440',
    #'EURCHF1440', 
#'CHFJPY1440',
    #'CADCHF1440', 
    #'AUDCHF1440',
    #'NZDCHF1440',
    #'EURGBP1440',
   #'GBPJPY1440',
    #'GBPCAD1440',
    #'GBPAUD1440',
    #'EURJPY1440', 
    #'EURCAD1440', 
    #'EURAUD1440', 
    #'EURNZD1440',
#'CADJPY1440', 
    #'AUDJPY1440',
    #'NZDJPY1440',
    #'AUDCAD1440',
#'NZDCAD1440',
#'AUDNZD1440'
                ]



headers = ['date_start', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]

In [154]:
shortest_row_currency = ''
lowest_number_rows = 1000000000000000000

for p in currency_list:
    
    current_currency_row = pd.read_csv('recent_data_daily/' + p + '.csv', names=headers  ) 
    current_currency_row = len( current_currency_row.index )
    
    if current_currency_row < lowest_number_rows:
        
        lowest_number_rows = current_currency_row
        shortest_row_currency = p
        
        
        
print(shortest_row_currency, lowest_number_rows)




USDCHF1440 8882


In [155]:
fundamental_data = pd.read_excel('investings_weekly_fundamental_data.xlsx', sheet_name=0  ) 
    
available_fundamental_timeframe  = fundamental_data['original date frame'].dt.strftime('%Y-%m-%d').values.tolist()
corrected_available_fundamental_timeframe =[]
for h in available_fundamental_timeframe:
            
    corrected_available_fundamental_timeframe.append(h.replace('-','.'))
    
    

In [156]:
# #loops through all currency, identify timeframes not in nzdcad and save them in global_available_timeframes list



shortest_row_currency = pd.read_csv('recent_data_daily/' + p + '.csv', names=headers  ) 

#this reverses the rows so as to have the recent timeframe at the top
#shortest_row_currency = shortest_row_currency.sort_index(axis=0,ascending=False)



shortest_row_currency_timeframes = shortest_row_currency['date_start']



shortest_row_currency_timeframes = shortest_row_currency_timeframes.values.tolist()


global_unavailable_timeframes = []
for x in currency_list:
    
    
    current_currency_data = pd.read_csv('recent_data_daily/' + x + '.csv', names=headers  )  
    
    current_currency_data_timeframes = current_currency_data['date_start']
   
    current_currency_data_timeframes = current_currency_data_timeframes.values.tolist()
   
    for y in shortest_row_currency_timeframes:
        
            if y not in current_currency_data_timeframes :
                
                global_unavailable_timeframes.append(y)
                

# this get the unique time frames because time frames could have been duplicated from different currencies 
global_unavailable_timeframes = np.unique(global_unavailable_timeframes)   


print(len(global_unavailable_timeframes))   
print(len(shortest_row_currency_timeframes))


0
8882


In [157]:
# removing the globally unavailable timeframes from the shortest row currency

for x in global_unavailable_timeframes:
    shortest_row_currency_timeframes.remove(x)
    
globally_available_time_frame = shortest_row_currency_timeframes
print(len(globally_available_time_frame))

8882


In [158]:
# loop through all currency pairs, get the needed rows from them and assemble all rows to form a combine_data_dataframe

combine_data_dataframe = pd.DataFrame(data={}, index = pd.Series(range(0,len(globally_available_time_frame))), )

for x in currency_list :
    
    
   
   





    current_currency_data = pd.read_csv('recent_data_daily/' + x + '.csv', names=headers  ) 
    
    # this sort the rows, so as to have the most recent time frame at the top
    current_currency_data = current_currency_data.sort_index(axis=0,ascending=False)
   
    current_currency_volume = []
    
    current_currency_class = []
    
    current_currency_day = []
    
    q = 0
    
    

    for y in globally_available_time_frame:
       
       
        for index, row in current_currency_data.iterrows():
            
           # print(y, row.date_start)
            if y  == row.date_start : 
               
                        
                #current_currency_volume.append(row.volume)            
                
                # condition when the curreny movement is a buy

                pips_corrector = 100000

                if x.endswith('jpy'):# use this to correct multiplier of jpy pairs
                    pips_corrector = 1000

                if row.open < row.close:

                    trend = 1


                    #classifying the risk level


                    if (row.open - row.low) == 0 : # no risk because open is equal to low

                        risk = 6

                    else : 

                        if (row.open - row.low)> (row.high - row.open):

                            risk = 1

                        if  (row.high - row.open) > (row.open - row.low):

                            risk = 2


                        if (row.high - row.open) /(row.open - row.low) >2 :

                            risk = 3

                        if (row.high - row.open) /(row.open - row.low) >4 :

                            risk = 4


                        if (row.high - row.open) /(row.open - row.low) >8 :

                            risk = 5

                        if (row.high - row.open) /(row.open - row.low) >16 :

                            risk = 6  

                    #classifying reward level

                    if (row.close - row.open)*pips_corrector > 0 :

                        reward = 1

                    if (row.close - row.open)*pips_corrector >200 :

                        reward = 2

                    if (row.close - row.open)*pips_corrector > 400 :

                        reward = 3


                    if (row.close - row.open)*pips_corrector > 800 :

                        reward = 4

                    if (row.close - row.open)*pips_corrector > 1600 :

                        reward = 5


                    if (row.close - row.open)*pips_corrector > 3200 :

                        reward = 6  




                # this is for the condition when the movement is a sell
                else: 

                    trend = -1

                    #classifying the risk level


                    if (row.high - row.open) == 0 : # no risk because open is equal to high

                        risk = 6

                    else : 

                        if (row.high - row.open)> (row.open - row.low):

                            risk = 1

                        if  (row.open - row.low) > (row.high - row.open):

                            risk = 2


                        if (row.open - row.low) /(row.high - row.open) >2 :

                            risk = 3

                        if (row.open - row.low) /(row.high - row.open) >4 :

                            risk = 4

                        if (row.open - row.low) /(row.high - row.open) >8 :

                            risk = 5

                        if (row.open - row.low) /(row.high - row.open) >16 :

                            risk = 6 


                     #classifying reward level

                    if (row.open - row.close)*pips_corrector > 0 :

                        reward = 1

                    if (row.open - row.close)*pips_corrector > 200 :

                        reward = 2

                    if (row.open - row.close)*pips_corrector > 400 :

                        reward = 3        

                    if (row.open - row.close)*pips_corrector > 800 :

                        reward = 4    

                    if (row.open - row.close)*pips_corrector > 1600 :

                        reward = 5 

                    if (row.open - row.close)*pips_corrector > 3200 :

                        reward = 6        
            
                #print(index)
                break 
            
        current_currency_class.append(trend*risk*reward)
        current_currency_volume.append(row.volume)   
        current_currency_day.append(datetime.date(int(row.date_start[0:4]), int(row.date_start[5:7]), int(row.date_start[8:10])).strftime('%w'))

            #attach the classification score value to the dataframe and train a model with the data
            #print(index)
    #print(current_currency_class)

            #combine_data_dataframe[x+'_class'] = current_currency_class
    
    
            
                        

    # this function takes a list and returns with a list of 5 list that is the data for the previous row
    
    def add_5_columns(real_column) :
        
        current_currency_column_add_on = [real_column, [], [], [], [], []]
        
        g = 1 
        
        while g < 6:
            
            current_currency_column_add_on[g] = current_currency_column_add_on[g-1].copy()
            current_currency_column_add_on[g].pop(0)
            current_currency_column_add_on[g].append(0)    
            g = g+1
            
        current_currency_column_add_on.pop(0)
        return current_currency_column_add_on
            
            
           

    combine_data_dataframe[ x +'_volume'] = current_currency_volume
    
    #volume_columns = add_5_columns(current_currency_volume)
    
    
    
    
    combine_data_dataframe[ x +'_class'] = current_currency_class
    
    
    combine_data_dataframe[ x +'_day'] = current_currency_day
    
    
    
    
    
    
    
    
    
    #low_columns = add_5_columns(current_currency_low)
    
   
    
    
    
    #combine_data_dataframe[ x +'_close'] = current_currency_close
    
   




    
    print(x)
    
    




USDCHF1440


In [159]:
#create and add the timeframe column to the combine_data_data_frame to be able to identify each rows

date_series = pd.Series(globally_available_time_frame)

# the code below convert it to panda date time object
date_series = pd.to_datetime(date_series)


#turning date_series to a data frame so as to be able to add it as the first column 
date_series_dataframe =  pd.DataFrame(data={}, index = pd.Series(range(0, len(globally_available_time_frame))), )
date_series_dataframe['date_start'] = date_series



    


#the two dataframes are combined along the y axis to form one dataframe with all information and date
combine_data_dataframe = pd.concat([ date_series_dataframe, combine_data_dataframe, ], axis=1)

combine_data_dataframe


,date_start,USDCHF1440_volume,USDCHF1440_class,USDCHF1440_day
0,1987-01-02,1401,12,5
1,1987-01-05,3301,16,1
2,1987-01-06,2251,-30,2
3,1987-01-07,3101,16,3
4,1987-01-08,3701,-12,4
...,...,...,...,...
8877,2021-09-21,34219,-8,2
8878,2021-09-22,45315,4,3
8879,2021-09-23,37632,-2,4
8880,2021-09-24,27809,2,5


In [160]:



fundamental_data = fundamental_data[[
                                #'eur_black', 
                                     #'eur_green',
                                     #'eur_red',
                                        #'gbp_green',
                                        #'gbp_red',
                                     #'usd_no_actual'
                                    ]]

#fundamental_data = fundamental_data.drop(columns=['original date frame', ])


fundamental_data = fundamental_data[::-1].reset_index(drop=True)

currency = ['usd','gbp','jpy','aud','chf','eur','cad','nzd',]



#print(fundamental_data)

combine_data_dataframe = pd.concat([ combine_data_dataframe, fundamental_data ], axis=1)
print(combine_data_dataframe)

     date_start  USDCHF1440_volume  USDCHF1440_class USDCHF1440_day
0    1987-01-02               1401                12              5
1    1987-01-05               3301                16              1
2    1987-01-06               2251               -30              2
3    1987-01-07               3101                16              3
4    1987-01-08               3701               -12              4
...         ...                ...               ...            ...
8877 2021-09-21              34219                -8              2
8878 2021-09-22              45315                 4              3
8879 2021-09-23              37632                -2              4
8880 2021-09-24              27809                 2              5
8881 2021-09-27                  5                 6              1

[8882 rows x 4 columns]


In [161]:
# copies the data frame so as to isolate the first 10 rows for prediction
combine_top_10_rows_dataframe = combine_data_dataframe.copy(deep=True)


combine_top_10_rows_dataframe = combine_top_10_rows_dataframe.head(25)

#saves the first 10 rows to excel

combine_top_10_rows_dataframe.to_excel("files/combine_top_10_rows_dataframe.xlsx")



#combine_data_dataframe = combine_data_dataframe.head( len(globally_available_time_frame) - 5 )

combine_data_dataframe.to_excel("files/combine_data_dataframe.xlsx") 
